In [0]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
 

Using TensorFlow backend.


# Model 1: One-Word-In, One-Word-Out Sequences

In [0]:
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result

In [0]:
# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

In [0]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [0]:
encoded

[2,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 2,
 14,
 15,
 1,
 16,
 17,
 18,
 1,
 3,
 19,
 20,
 21]

In [0]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 22


In [0]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
	sequence = encoded[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 24


In [0]:
sequences

[[2, 1],
 [1, 3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6, 7],
 [7, 8],
 [8, 9],
 [9, 10],
 [10, 11],
 [11, 12],
 [12, 13],
 [13, 2],
 [2, 14],
 [14, 15],
 [15, 1],
 [1, 16],
 [16, 17],
 [17, 18],
 [18, 1],
 [1, 3],
 [3, 19],
 [19, 20],
 [20, 21]]

In [0]:
# split into X and y elements
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

In [0]:
X

array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1,
       16, 17, 18,  1,  3, 19, 20])

In [0]:
y

array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1, 16,
       17, 18,  1,  3, 19, 20, 21])

In [0]:
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)

In [0]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 10)             220       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# fit network
model.fit(X, y, epochs=500, verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/500





 - 1s - loss: 3.0903 - acc: 0.0833
Epoch 2/500
 - 0s - loss: 3.0895 - acc: 0.0833
Epoch 3/500
 - 0s - loss: 3.0887 - acc: 0.1667
Epoch 4/500
 - 0s - loss: 3.0879 - acc: 0.1250
Epoch 5/500
 - 0s - loss: 3.0871 - acc: 0.1250
Epoch 6/500
 - 0s - loss: 3.0863 - acc: 0.1250
Epoch 7/500
 - 0s - loss: 3.0855 - acc: 0.1250
Epoch 8/500
 - 0s - loss: 3.0847 - acc: 0.1250
Epoch 9/500
 - 0s - loss: 3.0839 - acc: 0.1250
Epoch 10/500
 - 0s - loss: 3.0831 - acc: 0.1250
Epoch 11/500
 - 0s - loss: 3.0822 - acc: 0.1250
Epoch 12/500
 - 0s - loss: 3.0814 - acc: 0.1250
Epoch 13/500
 - 0s - loss: 3.0805 - acc: 0.1250
Epoch 14/500
 - 0s - loss: 3.0796 - acc: 0.1250
Epoch 15/500
 - 0s - loss: 3.0787 - acc: 0.1250
Epoch 16/500
 - 0s - loss: 3.0778 - acc: 0.1250
Epoch 17/500
 - 0s - loss: 3.0769 - acc: 0.1250
Epoch 18/500
 - 0s - loss: 3.0759 - acc: 0.1250
Epoch 19/500
 - 0s - loss: 3.0749 - acc: 0.

In [0]:
# evaluate
print(generate_seq(model, tokenizer, 'Jack', 6))


Jack and jill came tumbling after the


# Model 2: Line-by-Line Sequence

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [0]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [0]:
# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

In [0]:
# prepare the tokenizer on the source text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [0]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 22


In [0]:
# create line-based sequences
sequences = list()
for line in data.split('\n'):
	encoded = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(encoded)):
		sequence = encoded[:i+1]
		sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 21


In [0]:
sequences

[[2, 1],
 [2, 1, 3],
 [2, 1, 3, 4],
 [2, 1, 3, 4, 5],
 [2, 1, 3, 4, 5, 6],
 [2, 1, 3, 4, 5, 6, 7],
 [8, 9],
 [8, 9, 10],
 [8, 9, 10, 11],
 [8, 9, 10, 11, 12],
 [8, 9, 10, 11, 12, 13],
 [2, 14],
 [2, 14, 15],
 [2, 14, 15, 1],
 [2, 14, 15, 1, 16],
 [2, 14, 15, 1, 16, 17],
 [2, 14, 15, 1, 16, 17, 18],
 [1, 3],
 [1, 3, 19],
 [1, 3, 19, 20],
 [1, 3, 19, 20, 21]]

In [0]:
# pad input sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 7


In [0]:
sequences

array([[ 0,  0,  0,  0,  0,  2,  1],
       [ 0,  0,  0,  0,  2,  1,  3],
       [ 0,  0,  0,  2,  1,  3,  4],
       [ 0,  0,  2,  1,  3,  4,  5],
       [ 0,  2,  1,  3,  4,  5,  6],
       [ 2,  1,  3,  4,  5,  6,  7],
       [ 0,  0,  0,  0,  0,  8,  9],
       [ 0,  0,  0,  0,  8,  9, 10],
       [ 0,  0,  0,  8,  9, 10, 11],
       [ 0,  0,  8,  9, 10, 11, 12],
       [ 0,  8,  9, 10, 11, 12, 13],
       [ 0,  0,  0,  0,  0,  2, 14],
       [ 0,  0,  0,  0,  2, 14, 15],
       [ 0,  0,  0,  2, 14, 15,  1],
       [ 0,  0,  2, 14, 15,  1, 16],
       [ 0,  2, 14, 15,  1, 16, 17],
       [ 2, 14, 15,  1, 16, 17, 18],
       [ 0,  0,  0,  0,  0,  1,  3],
       [ 0,  0,  0,  0,  1,  3, 19],
       [ 0,  0,  0,  1,  3, 19, 20],
       [ 0,  0,  1,  3, 19, 20, 21]], dtype=int32)

In [0]:
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [0]:
X

array([[ 0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  2,  1],
       [ 0,  0,  0,  2,  1,  3],
       [ 0,  0,  2,  1,  3,  4],
       [ 0,  2,  1,  3,  4,  5],
       [ 2,  1,  3,  4,  5,  6],
       [ 0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  8,  9],
       [ 0,  0,  0,  8,  9, 10],
       [ 0,  0,  8,  9, 10, 11],
       [ 0,  8,  9, 10, 11, 12],
       [ 0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  2, 14],
       [ 0,  0,  0,  2, 14, 15],
       [ 0,  0,  2, 14, 15,  1],
       [ 0,  2, 14, 15,  1, 16],
       [ 2, 14, 15,  1, 16, 17],
       [ 0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  1,  3],
       [ 0,  0,  0,  1,  3, 19],
       [ 0,  0,  1,  3, 19, 20]], dtype=int32)

In [0]:
y

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [0]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 6, 10)             220       
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_4 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 1s - loss: 3.0911 - acc: 0.0952
Epoch 2/500
 - 0s - loss: 3.0894 - acc: 0.0000e+00
Epoch 3/500
 - 0s - loss: 3.0877 - acc: 0.0476
Epoch 4/500
 - 0s - loss: 3.0859 - acc: 0.0952
Epoch 5/500
 - 0s - loss: 3.0841 - acc: 0.0952
Epoch 6/500
 - 0s - loss: 3.0822 - acc: 0.0952
Epoch 7/500
 - 0s - loss: 3.0803 - acc: 0.0952
Epoch 8/500
 - 0s - loss: 3.0783 - acc: 0.0952
Epoch 9/500
 - 0s - loss: 3.0762 - acc: 0.0952
Epoch 10/500
 - 0s - loss: 3.0740 - acc: 0.0952
Epoch 11/500
 - 0s - loss: 3.0717 - acc: 0.0952
Epoch 12/500
 - 0s - loss: 3.0693 - acc: 0.0952
Epoch 13/500
 - 0s - loss: 3.0667 - acc: 0.0952
Epoch 14/500
 - 0s - loss: 3.0640 - acc: 0.0952
Epoch 15/500
 - 0s - loss: 3.0611 - acc: 0.0952
Epoch 16/500
 - 0s - loss: 3.0581 - acc: 0.0952
Epoch 17/500
 - 0s - loss: 3.0548 - acc: 0.0952
Epoch 18/500
 - 0s - loss: 3.0513 - acc: 0.0952
Epoch 19/500
 - 0s - loss: 3.0476 - acc: 0.0952
Epoch 20/500
 - 0s - loss: 3.0436 - acc: 0.0952
Epoch 21/500
 - 0s - loss: 3.0393 - acc: 0.09

In [0]:
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, 'Jack', 4))
print(generate_seq(model, tokenizer, max_length-1, 'Jill', 4))

Jack fell down and broke
Jill jill came tumbling after


# Model 3: Two-Words-In, One-Word-Out Sequence

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [0]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text
 

In [0]:
# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [0]:
encoded

[2,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 2,
 14,
 15,
 1,
 16,
 17,
 18,
 1,
 3,
 19,
 20,
 21]

In [0]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 22


In [0]:
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 23


In [0]:
sequences

[[2, 1, 3],
 [1, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9],
 [8, 9, 10],
 [9, 10, 11],
 [10, 11, 12],
 [11, 12, 13],
 [12, 13, 2],
 [13, 2, 14],
 [2, 14, 15],
 [14, 15, 1],
 [15, 1, 16],
 [1, 16, 17],
 [16, 17, 18],
 [17, 18, 1],
 [18, 1, 3],
 [1, 3, 19],
 [3, 19, 20],
 [19, 20, 21]]

In [0]:
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 3


In [0]:
sequences

array([[ 2,  1,  3],
       [ 1,  3,  4],
       [ 3,  4,  5],
       [ 4,  5,  6],
       [ 5,  6,  7],
       [ 6,  7,  8],
       [ 7,  8,  9],
       [ 8,  9, 10],
       [ 9, 10, 11],
       [10, 11, 12],
       [11, 12, 13],
       [12, 13,  2],
       [13,  2, 14],
       [ 2, 14, 15],
       [14, 15,  1],
       [15,  1, 16],
       [ 1, 16, 17],
       [16, 17, 18],
       [17, 18,  1],
       [18,  1,  3],
       [ 1,  3, 19],
       [ 3, 19, 20],
       [19, 20, 21]], dtype=int32)

In [0]:
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [0]:
X

array([[ 2,  1],
       [ 1,  3],
       [ 3,  4],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [11, 12],
       [12, 13],
       [13,  2],
       [ 2, 14],
       [14, 15],
       [15,  1],
       [ 1, 16],
       [16, 17],
       [17, 18],
       [18,  1],
       [ 1,  3],
       [ 3, 19],
       [19, 20]], dtype=int32)

In [0]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [0]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 2, 10)             220       
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_5 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 1s - loss: 3.0902 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 3.0895 - acc: 0.0000e+00
Epoch 3/500
 - 0s - loss: 3.0887 - acc: 0.1304
Epoch 4/500
 - 0s - loss: 3.0879 - acc: 0.1739
Epoch 5/500
 - 0s - loss: 3.0870 - acc: 0.1739
Epoch 6/500
 - 0s - loss: 3.0861 - acc: 0.1739
Epoch 7/500
 - 0s - loss: 3.0852 - acc: 0.1739
Epoch 8/500
 - 0s - loss: 3.0842 - acc: 0.1739
Epoch 9/500
 - 0s - loss: 3.0833 - acc: 0.1739
Epoch 10/500
 - 0s - loss: 3.0823 - acc: 0.1739
Epoch 11/500
 - 0s - loss: 3.0813 - acc: 0.1739
Epoch 12/500
 - 0s - loss: 3.0803 - acc: 0.1739
Epoch 13/500
 - 0s - loss: 3.0793 - acc: 0.1739
Epoch 14/500
 - 0s - loss: 3.0782 - acc: 0.1739
Epoch 15/500
 - 0s - loss: 3.0771 - acc: 0.1739
Epoch 16/500
 - 0s - loss: 3.0759 - acc: 0.1739
Epoch 17/500
 - 0s - loss: 3.0748 - acc: 0.1739
Epoch 18/500
 - 0s - loss: 3.0736 - acc: 0.1739
Epoch 19/500
 - 0s - loss: 3.0723 - acc: 0.1739
Epoch 20/500
 - 0s - loss: 3.0711 - acc: 0.1739
Epoch 21/500
 - 0s - loss: 3.0697 - acc: 

In [0]:
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 5))
print(generate_seq(model, tokenizer, max_length-1, 'And Jill', 3))
print(generate_seq(model, tokenizer, max_length-1, 'fell down', 5))
print(generate_seq(model, tokenizer, max_length-1, 'pail of', 5))

Jack and jill came tumbling after hill
And Jill came tumbling after
fell down and broke his crown and
pail of water jack fell down and


# https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/